In [9]:
import os
from transformers import AutoTokenizer, AutoModel, BitsAndBytesConfig, AutoModelForCausalLM
import torch
from tqdm.auto import tqdm
import gc
import re

In [24]:
base_path = '/projets/melodi/gsantoss/complex-llm/generated-prompts/conference/ont'


raw_prompts = {}

for p, d, fs in os.walk(base_path):
    for f in fs:
        if 'edas#ekaw' not in f:
            continue
        
        with open(os.path.join(p, f), 'r') as fl:
            raw_prompts[f] = fl.read()



In [3]:
model_id = 'meta-llama/Meta-Llama-3-8B-Instruct'
# model_id = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
# model_id = 'google/gemma-2-27b-it'
# model_id = "google/gemma-2b-it"
# model_id = "microsoft/Phi-3-mini-128k-instruct"
# model_id = "unsloth/llama-3-8b-bnb-4bit"


tokenizer = AutoTokenizer.from_pretrained(model_id)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)


model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map='auto',
    quantization_config=quantization_config,
    low_cpu_mem_usage=True,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [17]:
smp = '''<?xml version="1.0" encoding="UTF-8"?>
    <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns="http://knowledgeweb.semanticweb.org/heterogeneity/alignment#" xmlns:alext="http://exmo.inrialpes.fr/align/ext/1.0/" xmlns:align="http://knowledgeweb.semanticweb.org/heterogeneity/alignment#" xmlns:edoal="http://ns.inria.org/edoal/1.0/#" xmlns:xsd="http://www.w3.org/2001/XMLSchema#">
      <Alignment>
        <xml>yes</xml>
        <level>2EDOAL</level>
        <type>**</type>
        <onto1>
          <Ontology rdf:about="http://example.org/ontology1/" />
        </onto1>
        <onto2>
          <Ontology rdf:about="http://example.org/ontology2/" />
        </onto2>
        <map>
          <Cell>
            <entity1>
              <edoal:Class rdf:about="http://example.org/ontology1/AcceptedPaper" />
            </entity1>
            <entity2>
              <edoal:Relation>
                <edoal:and rdf:parseType="Collection">
                  <edoal:RelationDomainRestriction>
                    <edoal:Class rdf:about="http://example.org/ontology2/Paper" />
                  </edoal:RelationDomainRestriction>
                  <edoal:Relation>
                    <edoal:and rdf:parseType="Collection">
                      <edoal:RelationCoDomainRestriction>
                        <edoal:Class rdf:about="http://example.org/ontology2/Acceptance" />
                      </edoal:RelationCoDomainRestriction>
                      <edoal:Relation rdf:about="http://example.org/ontology2/hasAcceptance" />
                    </edoal:and>
                  </edoal:Relation>
                </edoal:and>
              </edoal:Relation>
            </entity2>
            <relation>=</relation>
            <measure rdf:datatype="http://www.w3.org/2001/XMLSchema#float">1.0</measure>
          </Cell>
        </map>
        <map>
          <Cell>
            <entity1>
               <edoal:compose rdf:parseType="Collection">
                 <edoal:Relation rdf:resource="http://example.org/ontology1/hasFirstName" />
                 <edoal:Relation rdf:resource="http://example.org/ontology1/hasLastName" />
               </edoal:compose>
            </entity1>
            <entity2>
              <edoal:Relation rdf:resource="http://example.org/ontology2/hasFullName" />
            </entity2>
            <relation>=</relation>
            <measure rdf:datatype="http://www.w3.org/2001/XMLSchema#float">1.0</measure>
          </Cell>
        </map>
        <map>
          <Cell>
            <entity1>
              <edoal:Class rdf:about="http://example.org/ontology1/Author" />
            </entity1>
            <entity2>
              <edoal:Class rdf:about="http://example.org/ontology2/Author" />
            </entity2>
            <relation>=</relation>
            <measure rdf:datatype="http://www.w3.org/2001/XMLSchema#float">1.0</measure>
          </Cell>
        </map>
        <map>
          <Cell>
            <entity1>
              <edoal:Class rdf:about="http://example.org/ontology1/Conference" />
            </entity1>
            <entity2>
              <edoal:Class rdf:about="http://example.org/ontology2/Conference" />
            </entity2>
            <relation>=</relation>
            <measure rdf:datatype="http://www.w3.org/2001/XMLSchema#float">1.0</measure>
          </Cell>
        </map>
        <map>
          <Cell>
            <entity1>
              <edoal:InverseRelation rdf:about="http://example.org/ontology1/isAuthorOf" />
            </entity1>
            <entity2>
              <edoal:Relation rdf:about="http://example.org/ontology2/hasAuthor" />
            </entity2>
            <relation>=</relation>
            <measure rdf:datatype="http://www.w3.org/2001/XMLSchema#float">1.0</measure>
          </Cell>
        </map>
        <map>
          <Cell>
            <entity1>
              <edoal:Relation rdf:about="http://example.org/ontology1/isPresentedAt" />
            </entity1>
            <entity2>
              <edoal:Relation rdf:about="http://example.org/ontology2/isSubmittedTo" />
            </entity2>
            <relation>=</relation>
            <measure rdf:datatype="http://www.w3.org/2001/XMLSchema#float">1.0</measure>
          </Cell>
        </map>
      </Alignment>
    </rdf:RDF>
    '''

In [37]:
for k, v in raw_prompts.items():
    if 'nq-s1-s2' not in k:
        continue
    print(k)
    txt = v
    break

prompt#edas#ekaw#c-country#nq-s1-s2.txt


In [32]:
li = txt.rfind('<result>')
lo = txt.rfind('</result>')

txt = txt[:li + len('<result>')] + smp + txt[lo:]

In [38]:
messages = [
    {"role": "system", "content": "You are an Ontology Alignment expert. You are able to align two ontologies by creating a file in EDOAL format containing the result alignments. You are able to produce complex alignments that are those involving multiple entities and relationships in a n:m cardinality. The user will provide you with two ontologies and you respond with the EDOAL file containing the alignments. You don't need to explain yourself. Just give as response the resulting file without saying anything."},
    {"role": "user", "content": txt},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

print(input_ids.shape)

torch.Size([1, 3378])


In [39]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_new_tokens=2 * 1024,
        eos_token_id=terminators,
        do_sample=False,
        temperature=None,
        top_p=None,

    )
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


<?xml version="1.0" encoding="UTF-8"?>
<rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns="http://knowledgeweb.semanticweb.org/heterogeneity/alignment#" xmlns:alext="http://exmo.inrialpes.fr/align/ext/1.0/" xmlns:align="http://knowledgeweb.semanticweb.org/heterogeneity/alignment#" xmlns:edoal="http://ns.inria.org/edoal/1.0/#" xmlns:xsd="http://www.w3.org/2001/XMLSchema#">
  <Alignment>
    <xml>yes</xml>
    <level>2EDOAL</level>
    <type>**</type>
    <onto1>
      <Ontology rdf:about="http://edas" />
    </onto1>
    <onto2>
      <Ontology rdf:about="http://ekaw" />
    </onto2>
    <map>
      <Cell>
        <entity1>
          <edoal:Class rdf:about="http://edas" />
        </entity1>
        <entity2>
          <edoal:Class rdf:about="http://ekaw#Organisation" />
        </entity2>
        <relation>=</relation>
        <measure rdf:datatype="http://www.w3.org/2001/XMLSchema#float">1.0</measure>
      </Cell>
    </map>
    <map>
      <Cell>
        <entity1